In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import scipy.stats
import scipy.optimize
from scipy.sparse import csr_matrix
import scipy.signal

import cv2

import os
import functools
import itertools as it
from glob import glob

In [ ]:
PIX_SIZE = .00112

DIR = os.path.join(os.getenv('HOME'), 'beam_sim')
TAG = 'protons'

In [ ]:
# get variables from config file
f_cfg = np.load('{}/{}_config.npz'.format(DIR, TAG))

RES_X, RES_Y = f_cfg['res']
FPS = f_cfg['fps']
PARTICLES = f_cfg['particles']

f_cfg.close()

LIM_X = PIX_SIZE * RES_X / 2
LIM_Y = PIX_SIZE * RES_Y / 2

# I. Measuring Background #

In [ ]:
histograms = {}
n_frames = {}

NOISE = {}

for fname in os.listdir(os.path.join(DIR, 'bg')):
    p = fname.split('_')[0]
    if not p in histograms:
        histograms[p] = 0
        n_frames[p] = 0
    
    f = np.load(os.path.join(DIR, 'bg', fname))
    
    histograms[p] += csr_matrix((np.ones(f['x'].size), (f['x'], f['y'])), shape=(RES_X,RES_Y))
    n_frames[p] += 1
    
    f.close()
    
downsample = 97

for p,h in histograms.items():
    downscale = h.toarray().reshape(RES_X//downsample, downsample, RES_Y//downsample, downsample).sum((1,3))
    downscale = scipy.signal.convolve2d(downscale, np.ones((3,3))/9, mode='same', boundary='symm')
    NOISE[p] = cv2.resize(downscale, (RES_Y, RES_X)) / n_frames[p] / downsample**2
    
    plt.figure()
    plt.imshow(NOISE[p].transpose(), cmap='viridis')
    plt.colorbar()

# II. Sorting the Data #

We first need to sort the phone data into spills by time.  But this first requires doing drift correction:

In [ ]:
class Spill(dict):
    def __init__(self):
        pass
    
    def append(self, phone, t):
        if not phone in self.keys():
            self[phone] = [t]
        else:
            self[phone].append(t)
            
    def get_file(phone, t, filetype='cluster'):
        return np.load(os.path.join(DIR, TAG, phone, filetype, '{}.npz'.format(t)))
    
    def histogram(self, phone, downsample=4):
        hist = 0
        
        for t in self[phone]:
            f = Spill.get_file(phone, t, filetype='cluster')
            res_y_down = RES_Y // downsample
            res_x_down = RES_X // downsample
            hist += np.histogram2d(f.f.y, f.f.x, bins=(res_y_down, res_x_down), range=((0, RES_Y),(0, RES_X)))[0]
            f.close()
        return hist

In [ ]:
phones_all = []
t_nominal = []
t_corr = []

prefix_splits = PREFIX.count('_')

for fname in glob(os.path.join(DIR, TAG, '*', 'cluster', '*')):
    head, base = os.path.split(fname)
    head, _ = os.path.split(head)
    head, phone = os.path.split(head)
    
    t = int(base[:-4])
    
    # in case this is run multiple times, we use the original timestamp saved in the raw file
    f = np.load(os.path.join(DIR, 'cluster', fname))
    
    phones_all.append(iphone)
    t_nominal.append(f.f.t)
    t_corr.append(t)
    f.close()
    
tmin = min(t_nominal)
tmax = max(t_nominal)

plt.title('Image times')
plt.hist(t_nominal, bins=100, range=(tmin, tmin+600000));

In [ ]:
sorting = np.argsort(t_nominal)
t_nominal_sorted = np.array(t_nominal)[sorting]
t_corr_sorted = np.array(t_corr)[sorting]
phones_sorted = np.array(phones_all)[sorting]

# pick a phone to which we map the other phones' coordinates
PHONE_LIST = sorted(list(set(phones_all))) # remove copies
ROOT = PHONE_LIST[0]
OTHERS = PHONE_LIST.copy()
OTHERS.remove(ROOT)
print(PHONE_LIST)
print(ROOT)

COMBINATIONS = [list(map(frozenset, it.combinations(PHONE_LIST, i))) for i in range(len(PHONE_LIST)+1)]

In [ ]:
# find the average number of hits when the beam is on

p_arr = np.array(phones_sorted)
t_nominal_arr = np.array(t_nominal_sorted)
t_corr_arr = np.array(t_corr_sorted)

median_flux = {}

for iphone in PHONE_LIST:
    n_hits = []
    t_nominal_iphone = t_nominal_arr[p_arr == iphone]
    t_corr_iphone = t_corr_arr[p_arr == iphone]
    for t in t_corr_iphone:
        f = Spill.get_file(iphone, t, 'cluster')
        n_hits.append(len(f.f.x))
    plt.hist(n_hits, bins=50, histtype='step', label=iphone[:6])
    plt.xlabel('Hits per frame')
    
    # now exclude the first and last frame of each beam dump
    t_diffs = np.hstack([[1e5], np.diff(t_nominal_iphone), [1e5]]) 
    
    flux_full = np.array(n_hits)[(t_diffs[1:] < 1.5e3 / FPS) & (t_diffs[:-1] < 1.5e3 / FPS)]
    
    median_flux[iphone] = np.median(flux_full) 
    if noise:
        median_flux[iphone] -= NOISE[iphone].sum()
    
plt.legend(loc='upper left');

print(median_flux)

In [ ]:
# now get the estimated start/end intervals for each spill, by phone

spill_intervals = {}

for iphone in PHONE_LIST:
    t_corr_iphone = t_corr_arr[p_arr == iphone]
    t_nominal_iphone = t_nominal_arr[p_arr == iphone]
    t_diffs = np.hstack([[1e5], np.diff(t_nominal_iphone), [1e5]])
    
    t_len = t_corr_iphone.size
    start_args = np.arange(t_len)[(t_diffs[1:] <= 5e3) & (t_diffs[:-1] > 5e3)]
    end_args = np.arange(t_len)[(t_diffs[1:] > 5e3) & (t_diffs[:-1] <= 5e3)]
    
    t_start_corr = []
    t_end_corr = []
    
    bg = NOISE[iphone].sum() if NOISE else 0
    
    for arg in start_args:
        f = Spill.get_file(iphone, t_corr_iphone[arg], 'cluster')
        t_start_corr.append(t_nominal_iphone[arg] + 1000 / FPS * (1 - (len(f.f.x) - bg) / median_flux[iphone]))
        f.close()
        
    for arg in end_args: 
        f = Spill.get_file(iphone, t_corr_iphone[arg], 'cluster')
        t_end_corr.append(t_nominal_iphone[arg] + 1000 / FPS * (len(f.f.x) - bg) / median_flux[iphone])
        f.close()
        
    spill_intervals[iphone] = np.vstack([t_start_corr, t_end_corr])
    

In [ ]:
# now we clean these up a bit
    
# first get rid of spills from extraneous triggering
spill_lengths = np.array([])
plt.xlabel('Calculated spill lengths (ms)');
for iphone, intervals in spill_intervals.items():
    l = intervals[1] - intervals[0]
    plt.hist(l, bins=20, histtype='step', label=iphone[:6]);
    plt.legend();
    spill_lengths = np.append(l, spill_lengths)

min_spill_length = np.mean(spill_lengths) / 2

for iphone, intervals in spill_intervals.items():
    l = intervals[1] - intervals[0]
    spill_intervals[iphone] = spill_intervals[iphone][:, l > min_spill_length]
    
# then we get rid of extra spills at the beginning
first_spills = [interval[0][0] for interval in spill_intervals.values()]
min_spills = min([interval.shape[1] for interval in spill_intervals.values()])

for iphone, intervals in spill_intervals.items():
    spill_intervals[iphone] = spill_intervals[iphone][:, intervals[0] > max(first_spills) - 5000][:min_spills] 

In [ ]:
# finally get the linear coefficients
times_root = spill_intervals[ROOT].flatten()
times_root.sort()

m = {ROOT: 1}
b = {ROOT: 0}

for other in OTHERS:
    times_other = spill_intervals[other].flatten()
    times_other.sort()
    mi,bi,r,p,s = scipy.stats.linregress(times_other, times_root)
    print(mi,bi)
    m[other] = mi
    b[other] = bi
    
    plt.xlabel(other[:6])
    plt.ylabel(ROOT[:6])
    plt.plot(times_other[-8:], times_other[-8:], 'r.', markersize=5, label='Nominal')
    plt.plot(times_other[-8:], times_root[-8:], 'b.', markersize=5, label='Actual')
    plt.plot(times_other[-8:], mi * times_other[-8:] + bi, lw=0.5, label='Correction')
    plt.legend()
    plt.show()

In [ ]:
# now rename to reflect the adjusted times
times = []
phones = []

for fname in os.listdir(DIR + '/cluster/'):
    parts = (fname.split('.')[0]).split('_')
    if len(parts) != 3: continue
    iphone = parts[-2][1:]
    t = int(parts[-1][1:])
    f = np.load(os.path.join(DIR, 'cluster', fname))
    t_corr = int(m[iphone] * f.f.t + b[iphone])
    f.close()
    
    phones.append(iphone)
    times.append(t_corr)
    
    f_full = os.path.join(DIR, 'cluster', fname)
    os.rename(f_full, f_full.replace(str(t), str(t_corr)))

In [ ]:
# redo sorting
sorting = np.argsort(times)
t_sorted = np.array(times)[sorting]
phones_sorted = np.array(phones)[sorting]

dt = np.diff(t_sorted)

plt.title(r'$\Delta t$ between images')
plt.hist(dt, bins=50);

In [ ]:
# next, we separate the various spills in the files

def get_tuple(ientry):
    return phones_sorted[ientry], t_sorted[ientry]

n_phones = len(set(phones_sorted))
spills = []
spill_lengths = []
imin = 0

ispill = Spill()
iphone, t = get_tuple(0)
ispill.append(iphone, t)

for i, delta in enumerate(dt):
    iphone, t = get_tuple(i+1)
    if delta > 5000:
        spill_lengths.append(t_sorted[i] - t_sorted[imin])
        imin = i+1
        spills.append(ispill)
        # create a new spill
        ispill = Spill()
    
    ispill.append(iphone, t)

spill_lengths.append(max(t_sorted) - t_sorted[imin])
spills.append(ispill)

# III. Alignment #

### Truth values ###

We need coordinates that will be useful for finding the true mapping from sensor to lab.  In Eulerian angles, we have:

$x_l=(x_s+x_0)(\cos\psi\cos\theta\cos\phi - \sin\psi\sin\phi) - (y_s+y_0)(\sin\psi\cos\theta\cos\phi -\cos\psi\sin\phi)$
$y_l=(x_s+x_0)(\cos\psi\cos\theta\sin\phi + \sin\psi\cos\phi) - (y_s+y_0)(\sin\psi\cos\theta\sin\phi +\cos\psi\cos\phi)$

First, we can redefine the offsets in terms of the lab frame:

$x_l=x_{l0} + x_s(\cos\psi\cos\theta\cos\phi - \sin\psi\sin\phi) - y_s(\sin\psi\cos\theta\cos\phi -\cos\psi\sin\phi)$
$y_l=y_{l0} + x_s(\cos\psi\cos\theta\sin\phi + \sin\psi\cos\phi) - y_s(\sin\psi\cos\theta\sin\phi +\cos\psi\cos\phi)$

Since the phones are more or less aligned, we expect that $\psi\approx -\phi$.  If we write $\psi=-\phi+\delta$, then in matrix notation:

$
\begin{pmatrix}
x \\
y
\end{pmatrix}
= \begin{pmatrix}
x_{l0} \\
y_{l0}
\end{pmatrix}
+
\begin{pmatrix}
\cos\phi & -\sin\phi \\
\sin\phi & \cos\phi
\end{pmatrix}
\begin{pmatrix}
\cos\theta & 0 \\
0 & 1
\end{pmatrix}
\begin{pmatrix}
\cos\phi & \sin\phi \\
-\sin\phi & \cos\phi
\end{pmatrix}
\begin{pmatrix}
\cos\delta & -\sin\delta \\
\sin\delta & \cos\delta
\end{pmatrix}
\begin{pmatrix}
x_s \\
y_s
\end{pmatrix}
$

The rightmost of these four transformations is a rotation in the lab space, whereas the others compress the sensor coordinates about some axis.  Multiplying these matrices together and simplifying, we see that:

$
\begin{pmatrix}
\cos\phi & -\sin\phi \\
\sin\phi & \cos\phi
\end{pmatrix}
\begin{pmatrix}
\cos\theta & 0 \\
0 & 1
\end{pmatrix}
\begin{pmatrix}
\cos\phi & \sin\phi \\
-\sin\phi & \cos\phi
\end{pmatrix} =
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
- (1 - \cos\theta)
\begin{pmatrix}
\cos\phi & \sin\phi
\end{pmatrix}
\begin{pmatrix}
\cos\phi \\ \sin\phi
\end{pmatrix}
$

$= \mathbf{1} - \mathbf{v}\cdot\mathbf{v}^T$

for some vector $\mathbf{v}$ with components between 0 and 1.  While we do not know the offset, we can expect that $\delta\approx 0$ and $\mathbf{v} \approx \mathbf{0}$

Furthermore,

$ (\mathbf{1} - \mathbf{v}\cdot\mathbf{v}^T)^{-1} = \mathbf{1} + \frac{\mathbf{v}\cdot\mathbf{v}^T}{\det(\mathbf{1} - \mathbf{v}\cdot\mathbf{v}^T)}$

and so we can model the combined effect of two of these transformations as:

$ UV^{-1} = \mathbf{1} - \mathbf{u}\cdot\mathbf{u}^T + \mathbf{v'}\cdot\mathbf{v'}^T + O(\mathbf{u}^2, \mathbf{v'}^2) \\
= \mathbf{1} - \begin{pmatrix}
u_x & u_{xy} \\
u_{xy} & u_y
\end{pmatrix} $

### Finding the mapping parameters ###

We can find a coarse estimate of $dx$ and $dy$ by comparing beam profiles.

In [ ]:
hists = {p:0 for p in PHONE_LIST}
downsample=97
noise_grids = {p: NOISE[p].reshape(RES_X//downsample, downsample, RES_Y//downsample, downsample).sum((1,3)).transpose() \
               for p in PHONE_LIST}

for spl in spills:
    for p in PHONE_LIST:
        hist_i = spl.histogram(p, downsample=downsample)
        hists[p] += hist_i - noise_grids[p]
        
for p in hists:
    plt.figure()
    plt.title(r'Flux in particles / (pixel $\cdot$ s): {}'.format(p[:6]))
    plt.imshow(hists[p] / downsample**2 / len(spills) / 4.2, cmap='plasma', origin='lower', extent=[0, RES_X, 0, RES_Y]);
    plt.colorbar();

In [ ]:
def chisq_offset(hist1, hist2, dx, dy):
    if dy > 0:
        hist1 = hist1[dy:,:]
        hist2 = hist2[:-dy, :]
    elif dy < 0:
        hist1 = hist1[:dy,:]
        hist2 = hist2[-dy:, :]
    
    if dx > 0:
        hist1 = hist1[:,dx:]
        hist2 = hist2[:,:-dx]
    elif dx < 0:
        hist1 = hist1[:, :dx]
        hist2 = hist2[:, -dx:]
        
    return np.sum((hist1/hist1.mean() - hist2/hist2.mean())**2/(hist1 + hist2)) / hist1.size

min_intersection = 5
shape = list(hists.values())[0].shape
chisq_grids = {c: np.array([[chisq_offset(hists[sorted(c)[0]], hists[sorted(c)[1]], x, y) \
                       for x in np.arange(-shape[1]+min_intersection, shape[1]-min_intersection+1)] \
                       for y in np.arange(-shape[0]+min_intersection, shape[0]-min_intersection+1)]) \
               for c in map(tuple, map(sorted, COMBINATIONS[2]))}
                       
sx = downsample*(shape[1] - 2*min_intersection)*PIX_SIZE 
sy = downsample*(shape[0] - 2*min_intersection)*PIX_SIZE

dxy_all = {}

for c, grid in chisq_grids.items():
    plt.figure()
    plt.title(r'$\chi^2$: ({}, {})'.format(c[0][:6], c[1][:6]))
    plt.xlabel('dx (mm)')
    plt.ylabel('dy (mm)')
    plt.imshow(grid, cmap='Spectral', norm=LogNorm(), extent=[-sx, sx, -sy, sy], origin='lower');
    plt.colorbar();

    iy, ix = np.unravel_index(np.argmin(grid), grid.shape)

    dxy_all[c] = (ix / (grid.shape[1]-1) * 2 * sx - sx, iy / (grid.shape[0]-1) * 2 * sy - sy)

In [ ]:
# now pick a root frame to map coordinates
furthest = {}

for c,xy in dxy_all.items():
    dsq = xy[0]**2 + xy[1]**2
    for p in c:
        if not p in furthest or dsq > furthest[p]:
            furthest[p] = dsq 
    
ROOT = max(furthest.keys(), key=(lambda key: furthest[key]))
OTHERS = [p for p in PHONE_LIST if p is not ROOT]
    
print(furthest)
print('ROOT:', ROOT)
print('OTHERS:', OTHERS)

dxy_coarse = {}
for c,xy in dxy_all.items():
    if ROOT == c[0]:
        dxy_coarse[c[1]] = xy
    elif ROOT == c[1]:
        dxy_coarse[c[0]] = (-xy[0], -xy[1])
        
print(dxy_coarse)

In [ ]:
# we convert the truth results to these new coordinates with the given ROOT
ftruth = np.load('{}/{}_truth.npz'.format(DIR, PREFIX))
print(ftruth.f.hwid, ROOT)

fig = plt.figure()
ax = fig.gca()

# first draw beam profile
spot_size = 6
beam_xy = scipy.stats.multivariate_normal(mean=[0,0], cov=spot_size/2)
n_particles=5e5
sz = 1.5*spot_size
x = np.linspace(-sz/2, sz/2, 100)
y = np.linspace(-sz/2, sz/2, 100)
pos = np.dstack(np.meshgrid(x,y))
intensity = n_particles*beam_xy.pdf(pos)
plt.imshow(intensity, extent=2*[-sz/2, sz/2], cmap='plasma', origin='lower')
plt.colorbar()

# next add rectangles for each phone
n_phones = ftruth.f.hwid.size

rects = np.repeat(np.array([[
        [-LIM_X, -LIM_Y],
        [LIM_X, -LIM_Y],
        [LIM_X, LIM_Y],
        [-LIM_X, LIM_Y]]]), n_phones, axis=0)

# do rotations

cos_phi = np.cos(ftruth.f.phi)
sin_phi = np.sin(ftruth.f.phi)
phi_mat = np.array([[cos_phi, -sin_phi], [sin_phi, cos_phi]])

theta_mat = np.array([
    [np.cos(ftruth.f.theta), np.zeros(n_phones)],
    [np.zeros(n_phones), np.ones(n_phones)]
    ])

cos_psi = np.cos(ftruth.f.psi)
sin_psi = np.sin(ftruth.f.psi)
psi_mat = np.array([[cos_psi, -sin_psi], [sin_psi, cos_psi]])

# need to rotate to frame of root phone for the right "truth" coords

root_idx = np.argwhere(ftruth.f.hwid == ROOT)[0]
delta0 = ftruth.f.phi[root_idx] + ftruth.f.psi[root_idx]
cos_delta = np.cos(delta0)[0]
sin_delta = np.sin(delta0)[0]
delta_mat = np.array([[cos_delta, sin_delta], [-sin_delta, cos_delta]])

xy_lab_truth = dict(zip(ftruth.f.hwid, np.dstack([ftruth.f.x, ftruth.f.y])[0]))

for iphone, phone in enumerate(ftruth.f.hwid):

    for corner in range(4):
        psi_rot_c = np.matmul(psi_mat[:,:,iphone], rects[iphone][corner])
        theta_rot_c = np.matmul(theta_mat[:,:,iphone], psi_rot_c)
        rects[iphone][corner] = np.matmul(phi_mat[:,:,iphone], theta_rot_c) + xy_lab_truth[phone]


phi_adj = np.mod(ftruth.f.phi, np.pi) - np.pi # same transformation mod pi

v_truth = dict(zip(ftruth.f.hwid, \
        np.sqrt(1-np.cos(ftruth.f.theta)).reshape(-1,1) * np.dstack([np.cos(phi_adj), np.sin(phi_adj)])[0]))
vinv_ROOT = v_truth[ROOT]/np.sqrt(1 - np.inner(v_truth[ROOT], v_truth[ROOT]))
scale_root = np.eye(2) + np.outer(vinv_ROOT, vinv_ROOT)
scale_mats = {p: np.eye(2) - np.outer(v_truth[p], v_truth[p]) for p in PHONE_LIST}

print("XY:")
print("abs")
print(np.array(list(xy_lab_truth.values())))
dxy_truth = {p: functools.reduce(np.matmul, [scale_root, delta_mat, xy_lab_truth[p] - xy_lab_truth[ROOT]]) for p in PHONE_LIST}
print("diff")
print(np.array(list(dxy_truth.values())))
print()

print("phi:")
print(ftruth.f.phi)
phi_lab_truth = dict(zip(ftruth.f.hwid, ftruth.f.phi + ftruth.f.psi))
print("abs")
print(np.array(list(phi_lab_truth.values())))
dphi_truth = {p: phi_lab_truth[p] - phi_lab_truth[ROOT] for p in PHONE_LIST}
print("diff")
print(np.array(list(dphi_truth.values())))
print()

print("u:")
u_truth = {p: np.array([v_truth[p][0]**2 - vinv_ROOT[0]**2, \
                        v_truth[p][0] * v_truth[p][1] - vinv_ROOT[0] * vinv_ROOT[1], \
                        v_truth[p][1]**2 - vinv_ROOT[1]**2]) \
           for p in PHONE_LIST}
#u_truth = {p: np.dot(scale_root, scale_mats[p]) for p in PHONE_LIST}
print(np.array(list(u_truth.values())))

align_truth = {p: np.hstack([dxy_truth[p], dphi_truth[p], u_truth[p]]) for p in PHONE_LIST}

polygons = [Polygon(rects[iphone]) for iphone in range(n_phones)]
p = PatchCollection(polygons, edgecolors='r', facecolors='none', linewidths=1)
ax.add_collection(p);

Now, we can do some finer adjustment which includes $\phi$.  Calculating the cdf for the nearest pixel from another frame:

$dp = 2\pi n r dr$

$1 - F(r) = \prod_{0}^{r}(1-dp) = \exp(-\int_{0}^{r}2\pi n r` dr`) = \exp(-\pi n r^2)$

$f(r) = 2\pi n r\exp(-\pi n r^2)$

$\langle r \rangle = \int_{0}^{\infty} 2 \pi n r^2 \exp(-\pi n r^2) dr$

$ = \frac{1}{\sqrt{\pi n}}\int_{-\infty}^{\infty}s^2 \exp(-s^2) ds$

$ = \frac{1}{2\sqrt{n}}$


In [ ]:
# methods for converting between frames

def sensor_map(xs, ys, dx=0, dy=0, phi=0, ux=0, uxy=0, uy=0):
    xy_s = np.array([xs, ys])
    phi_mat = np.array([
            [np.cos(phi), -np.sin(phi)], 
            [np.sin(phi), np.cos(phi)]
        ])
    
    scale = np.eye(2) - np.array([[ux, uxy],[uxy, uy]])
    
    return functools.reduce(np.dot, [scale, phi_mat, xy_s]) + np.array([[dx], [dy]])

def inverse_map(xlab, ylab, dx=0, dy=0, phi=0, ux=0, uxy=0, uy=0):
    xy_lab = np.array([xlab, ylab]) - np.array([[dx], [dy]])
    inv_phi_mat = np.array([
            [np.cos(phi), np.sin(phi)],
            [-np.sin(phi), np.cos(phi)]
        ])
    
    inv_scale = np.linalg.inv(np.eye(2) - np.array([[ux, uxy],[uxy, uy]]))
    
    return functools.reduce(np.dot, [inv_phi_mat, inv_scale, xy_lab])

In [ ]:
# method for sorting points for easier computation
def divide_points(x, y, ndivs, limits=None):
    if limits:
        xmin, xmax, ymin, ymax = limits
    else:
        xmin = np.amin(x)
        xmax = np.amax(x)
        ymin = np.amin(y)
        ymax = np.amin(y)
        
    divx = (xmax - xmin) / ndivs
    divy = (ymax - ymin) / ndivs
    
    return np.minimum((x - xmin) // divx, ndivs-1), np.minimum((y - ymin) // divy, ndivs-1)

Here, we introduce a suitable scoring function for alignments.

In [ ]:
def score_points(x1, y1, x2, y2, alpha=0.5, ndivs=13, hist_bins=100):
        
    xmin = min(np.amin(x1), np.amin(x2))
    xmax = max(np.amax(x1), np.amax(x2))
    ymin = min(np.amin(y1), np.amin(y2))
    ymax = max(np.amax(y1), np.amax(y2))
    
    divx = (xmax - xmin) / ndivs
    divy = (ymax - ymin) / ndivs
    
    # make interlaced cells
    x1_cells, y1_cells = divide_points(x1, y1, ndivs, limits=(xmin, xmax, ymin, ymax))
    x2_cells, y2_cells = divide_points(x2, y2, ndivs+1, limits=(xmin - divx/2, xmax + divx/2, ymin-divy/2, ymax+divy/2))
    
    total_score = 0
    survival_hist = np.zeros(hist_bins)
    
    for i,j in np.ndindex(ndivs, ndivs):
        group1 = (x1_cells == i) & (y1_cells == j) 
        if not group1.sum():
            continue
        
        group2 = ((x2_cells == i) | (x2_cells == i+1)) & ((y2_cells == j) | (y2_cells == j+1))
        if not group2.sum():
            continue
        
        rsquared_min = np.amin((x1[group1] - x2[group2].reshape(-1,1))**2 \
                               + (y1[group1] - y2[group2].reshape(-1,1))**2, axis=0)
    
        # this is just cdf for min Euclidean distance
        density = group2.sum() / (4 * divx * divy)
        survival = np.exp(-rsquared_min * np.pi * density)
        scores = np.maximum(survival - (1 - alpha), 0) / alpha
        total_score += np.sum(scores)
        survival_hist += np.histogram(survival, bins=hist_bins, range=(0,1))[0]
           
    return total_score / len(x1), survival_hist

In [ ]:
def calculate_overlap(fps, *t):
    diff = max(t) - min(t)
    return max(1 - fps * diff / 1000, 0)
        

def score_spills(p1, p2, *spills, dx=0, dy=0, phi=0, ux=0, uxy=0, uy=0, nmax=None, **kwargs):
    scores = 0
    total = 0
    n = 0
    
    for spl in spills:
        # to save time in coarse adjustment
        if nmax and n >= nmax: break
            
        spl1 = spl[p1].copy()
        spl2 = spl[p2].copy()
        
        t1 = spl1.pop(0)
        t2 = spl2.pop(0)
        
        while spl1 and spl2:
            if nmax and n >= nmax: break
                
            overlap = calculate_overlap(FPS, t1, t2)
            if overlap:
                
                n += 1

                f1 = Spill.get_file(p1, t1, 'cluster')
                f2 = Spill.get_file(p2, t2, 'cluster')
                       
                x1_sensor1 = (f1['x'] + 0.5) * PIX_SIZE - LIM_X
                y1_sensor1 = (f1['y'] + 0.5) * PIX_SIZE - LIM_Y
                x2_sensor2 = (f2['x'] + 0.5) * PIX_SIZE - LIM_X
                y2_sensor2 = (f2['y'] + 0.5) * PIX_SIZE - LIM_Y

                x2_sensor1, y2_sensor1 = sensor_map(x2_sensor2, y2_sensor2, dx=dx, dy=dy, phi=phi, ux=ux, uxy=uxy, uy=uy)
                
                # limit to hits that map to opposite sensor as well
                x1_sensor2, y1_sensor2 = inverse_map(x1_sensor1, y1_sensor1, dx=dx, dy=dy, phi=phi, ux=ux, uxy=uxy, uy=uy)

                intersect1 = (np.abs(x1_sensor2) < LIM_X) & (np.abs(y1_sensor2) < LIM_Y)
                intersect2 = (np.abs(x2_sensor1) < LIM_X) & (np.abs(y2_sensor1) < LIM_Y)

                x1_sensor1 = x1_sensor1[intersect1]
                y1_sensor1 = y1_sensor1[intersect1]
                x2_sensor1 = x2_sensor1[intersect2]
                y2_sensor1 = y2_sensor1[intersect2]

                score, _ = score_points(x1_sensor1, y1_sensor1, x2_sensor1, y2_sensor1, **kwargs)
                
                scores += overlap*score
                total += overlap
            
            if t1 > t2:
                t2 = spl2.pop(0)
            else:
                t1 = spl1.pop(0)

    return scores / total

print(score_spills(ROOT, OTHERS[1], *spills, alpha=0.3, ndivs=13, nmax=5))
%timeit score_spills(ROOT, OTHERS[1], *spills, alpha=0.3, ndivs=13, nmax=5)

In [ ]:
NDIVS = 14

We can now test our scoring function on the provided truth values.

In [ ]:
max_scores = {other: score_spills(ROOT, other, *spills[:5], dx=dxy_truth[other][0], dy=dxy_truth[other][1], phi=dphi_truth[other], \
             ux=u_truth[other][0], uxy=u_truth[other][1], uy=u_truth[other][2], alpha=.3) for other in OTHERS}
print(max_scores)

In [ ]:
def xy_grid(p1, p2, spills, dx, dy, delta, res, alpha=0.5, visualize=False, **kwargs):
    x_bins, y_bins = np.meshgrid(np.linspace(dx-delta, dx+delta, res), np.linspace(dy-delta, dy+delta, res))
    score_grid = np.array([[score_spills(p1, p2, *spills, dx=x, dy=y, alpha=alpha, ndivs=NDIVS, **kwargs) \
                   for x in np.linspace(dx-delta, dx+delta, res)] \
                  for y in np.linspace(dy-delta, dy+delta, res)])
    
    if visualize:
        plt.figure()
        plt.imshow(score_grid, extent=[dx-delta, dx+delta, dy-delta, dy+delta], cmap='seismic', \
                   origin='lower');
        plt.title('Truth ({}, {})'.format(p1[:6], p2[:6]))
        plt.xlabel('dx (mm)')
        plt.ylabel('dy (mm)')
        plt.colorbar()
        plt.show()
    return score_grid, x_bins, y_bins

# test with truth values
for other in OTHERS:
    xy_grid(ROOT, other, spills, *(dxy_truth[other]), .005, 13, phi=dphi_truth[other], \
            ux=u_truth[other][0], uxy=u_truth[other][1], uy=u_truth[other][2], \
            alpha=0.1, nmax=3, visualize=True);

In [ ]:
def phi_grid(p1, p2, spills, phi, delta, res, visualize=True, **kwargs):
    plt.figure()
    phi_bins = np.linspace(phi-delta, phi+delta, res)
    score_grid = np.array([score_spills(p1, p2, *spills, phi=phi_i, ndivs=NDIVS, **kwargs) for phi_i in phi_bins])
    
    if visualize:
        plt.plot(phi_bins, score_grid)
        plt.title('Truth ({}, {})'.format(p1[:6], p2[:6]))
        plt.xlabel(r'$\Delta\phi$')
        plt.ylabel('Score')
        plt.show()
    
    return score_grid, phi_bins
    
for other in OTHERS:
    phi_grid(ROOT, other, spills, dphi_truth[other], .01, 25, dx=dxy_truth[other][0], dy=dxy_truth[other][1], \
            ux=u_truth[other][0], uxy=u_truth[other][1], uy=u_truth[other][2], nmax=3, alpha=0.1);

In [ ]:
def get_max(grid, *bins):
    idx = np.unravel_index(np.argmax(grid), grid.shape)
    return tuple(b[idx] for b in bins)

Now with a combination of grid searches and Nelder-Mead optimization, we can find approximate $(x, y, \phi)$ coordinates.

In [ ]:
# start with increasingly precise grid searches in x,y,phi
xyphi = [{}]

def xy_phi_grid(p1, p2, spills, dx_guess, dy_guess, phi_guess, delta_xy, delta_phi, res_xy=11, res_phi=15, \
                visualize=None, **kwargs):

    if visualize and visualize != 'best' and visualize != 'all': 
        raise ValueError('"visualize" must be one of "best"/"all"')
    
    dx_result = dx_guess
    dy_result = dy_guess
    phi_result = phi_guess
    score_grid_best = None

    max_score = 0
    for phi_i in np.linspace(phi_guess-delta_phi, phi_guess+delta_phi, res_phi):
        if visualize=='all':
            print("phi={:.4f}".format(phi_i))
        score_grid, x_bins, y_bins = xy_grid(p1, p2, spills, dx_guess, dy_guess, delta_xy, res_xy, \
                                    phi=phi_i, visualize=(visualize=='all'), **kwargs)

        grid_max = np.amax(score_grid)
        if grid_max > max_score:
            max_score = grid_max

            dx_result, dy_result = get_max(score_grid, x_bins, y_bins)

            phi_result = phi_i
            
            score_grid_best = score_grid

    if visualize == 'best':
        plt.figure()
        plt.xlabel('dx (mm)')
        plt.ylabel('dy (mm)')
        plt.imshow(score_grid_best, extent=[dx_guess-delta_xy, dx_guess+delta_xy, dy_guess-delta_xy, dy_guess+delta_xy], \
                   cmap='seismic', origin='lower')
        plt.colorbar()
        plt.title(r'$\phi = {:.4}, ({}, {})$'.format(phi_result, p1[:6], p2[:6]))
        plt.show()
            
    return dx_result, dy_result, phi_result

In [ ]:
ITER = 0

# do the initial iteration separately
xyphi[ITER][OTHERS[0]] = xy_phi_grid(ROOT, OTHERS[0], spills, *dxy_coarse[OTHERS[0]], 0, 0.5, 0.15, 31, 15, alpha=0.3, nmax=3, visualize='all')

In [ ]:
ITER = 0
xyphi[ITER][OTHERS[1]] = xy_phi_grid(ROOT, OTHERS[1], spills, *dxy_coarse[OTHERS[1]], 0, 0.5, 0.15, 31, 15, alpha=0.3, nmax=3, visualize='best')

In [ ]:
ITER = 0
xyphi[ITER][OTHERS[2]] = xy_phi_grid(ROOT, OTHERS[2], spills, *dxy_coarse[OTHERS[2]], 0, 0.5, 0.15, 31, 15, alpha=0.3, nmax=3, visualize='best')

In [ ]:
ITER = 1

if len(xyphi) <= ITER:
    xyphi.append({})
else:
    xyphi[ITER] = {}

for other in OTHERS:
    xyphi[ITER][other] = xy_phi_grid(ROOT, other, spills, *xyphi[ITER-1][other], 0.06, .02, 15, 9, alpha=0.1, nmax=5, visualize='best')

In [ ]:
ITER = 2

if len(xyphi) <= ITER:
    xyphi.append({})
else:
    xyphi[ITER] = {}

for other in OTHERS:
    xyphi[ITER][other] = xy_phi_grid(ROOT, other, spills, *xyphi[ITER-1][other], .02, .005, 15, 9, alpha=0.1, nmax=5, visualize='best')

In [ ]:
# if we have a good enough initial guess, we can use an optimizer to find the maximum

ITER = 3

def f_xyphi(x, p1, p2, spills):
    return 1 - score_spills(p1, p2, *spills, dx=x[0], dy=x[1], phi=x[2], nmax=25, alpha=0.1)

if len(xyphi) <= ITER:
    xyphi.append({})
else:
    xyphi[ITER] = {}

for other in OTHERS:
    guess = np.array(xyphi[ITER-1][other])
    progress = [guess]
    res = scipy.optimize.minimize(f_xyphi, guess, args=(ROOT, other, spills), method='Nelder-Mead', callback = lambda xk: progress.append(xk))
    
    progress_arr = np.array(progress).transpose()
    
    xmin = np.amin(progress_arr[0])
    xmax = np.amax(progress_arr[0])
    ymin = np.amin(progress_arr[1])
    ymax = np.amax(progress_arr[1])
    
    plt.xlim(xmin-(xmax-xmin)/6, xmax+(xmax-xmin)/6)
    plt.ylim(ymin-(ymax-ymin)/6, ymax+(ymax-ymin)/6)
    
    plt.title(r'$(x, y, \phi)$ Convergence: ({}, {})'.format(ROOT[:6], other[:6]))
    plt.xlabel('dx (mm)')
    plt.ylabel('dy (mm)')
    plt.scatter(progress_arr[0], progress_arr[1], c=progress_arr[2], \
                s=160*np.arange(progress_arr.shape[1]+1)[::-1] / progress_arr.shape[1], cmap='winter', marker='x')
    plt.colorbar()
    plt.show()
    
    if res.success:
        print("Success!")
        dx, dy, phi = res.x
        xyphi[-1][other] = (dx, dy, phi)
    else: print("Failed:", res.message)

Now we can do a similar process for $U$.

In [ ]:
def u_grid(p1, p2, spills, ux, uxy, uy, delta, res, visualize=None, **kwargs):

    if visualize and visualize != 'best' and visualize != 'all': 
        raise ValueError('"visualize" must be one of "best"/"all"')
    
    ux_result = ux
    uxy_result = uxy
    uy_result = uy
    score_grid_best = None

    max_score = 0
    for uxy_i in np.linspace(uxy-delta, uxy+delta, res):
        
        score_grid = np.array([[score_spills(p1, p2, *spills, ux=ux_i, uxy=uxy_i, uy=uy_i, **kwargs) \
                   for ux_i in np.linspace(ux-delta, ux+delta, res)] \
                  for uy_i in np.linspace(uy-delta, uy+delta, res)])
        
        if visualize=='all':
            plt.figure()
            plt.imshow(score_grid, extent=[ux-delta, ux+delta, uy-delta, uy+delta], \
                       cmap='seismic', origin='lower')
            plt.colorbar()
            plt.title(r'$u_{xy} =$' + '{:.4}'.format(uxy_i))
            plt.show()

        grid_max = np.amax(score_grid)
        if grid_max > max_score:
            max_score = grid_max

            ux_bins, uy_bins = np.meshgrid(np.linspace(ux-delta, ux+delta, res), np.linspace(uy-delta, uy+delta, res))
            ux_result, uy_result = get_max(score_grid, ux_bins, uy_bins)

            uxy_result = uxy_i
            
            score_grid_best = score_grid

    if visualize == 'best':
        plt.figure()
        plt.imshow(score_grid_best, extent=[ux-delta, ux+delta, uy-delta, uy+delta], \
                   cmap='seismic', origin='lower')
        plt.colorbar()
        plt.title(r'$u_{xy} =$' + '{:.4}'.format(uxy_result))
        plt.xlabel(r'$u_x$')
        plt.ylabel(r'$u_y$')
        plt.show()
            
    return ux_result, uxy_result, uy_result

u_guess = [{}]

for other in OTHERS:
    u_guess[0][other] = u_grid(ROOT, other, spills, 0, 0, 0, 1e-2, 11, nmax=5, visualize='best', alpha=0.05, \
           dx=xyphi[-1][other][0], dy=xyphi[-1][other][1], phi=xyphi[-1][other][2],)

In [ ]:
ITER = 1

if len(u_guess) <= ITER:
    u_guess.append({})
else:
    u_guess[ITER] = {}

for other in OTHERS:
    u_guess[ITER][other] = u_grid(ROOT, other, spills, *u_guess[ITER-1][other], 4e-3, 11, nmax=5, alpha=0.05, \
           dx=xyphi[-1][other][0], dy=xyphi[-1][other][1], phi=xyphi[-1][other][2], visualize='best')

In [ ]:
ITER = 2

if len(u_guess) <= ITER:
    u_guess.append({})
else:
    u_guess[ITER] = {}

for other in OTHERS:
    u_guess[ITER][other] = u_grid(ROOT, other, spills, *u_guess[ITER-1][other], 1e-3, 11, nmax=7, alpha=0.05, \
           dx=xyphi[-1][other][0], dy=xyphi[-1][other][1], phi=xyphi[-1][other][2], visualize='best')

In [ ]:
ITER = 3

if len(u_guess) <= ITER:
    u_guess.append({})
else:
    u_guess[ITER] = {}

def f_u(x, p1, p2, spills, dx, dy, phi):
    return 1 - score_spills(p1, p2, *spills, dx=dx, dy=dy, phi=phi, \
                            ux=x[0], uxy=x[1], uy=x[2], 
                            nmax=10, alpha=0.03)

for other in OTHERS:
    guess = np.array(u_guess[ITER-1][other])
    progress = [guess]
    res = scipy.optimize.minimize(f_u, guess, args=(ROOT, other, spills, *xyphi[-1][other]), method='Nelder-Mead', \
                                  options={'xatol': 1e-5}, callback=lambda xk: progress.append(xk))
    
    progress_arr = np.array(progress).transpose()
    
    xmin = np.amin(progress_arr[0])
    xmax = np.amax(progress_arr[0])
    ymin = np.amin(progress_arr[2])
    ymax = np.amax(progress_arr[2])
    
    plt.xlim(xmin-(xmax-xmin)/6, xmax+(xmax-xmin)/6)
    plt.ylim(ymin-(ymax-ymin)/6, ymax+(ymax-ymin)/6)
    
    plt.title(r'$(u_x, u_y, u_{xy})$ Convergence: ' + '({}, {})'.format(ROOT[:6], other[:6]))
    plt.xlabel(r'$u_x$')
    plt.ylabel(r'$u_y$')
    plt.scatter(progress_arr[0], progress_arr[2], c=progress_arr[1], \
                s=160*np.arange(progress_arr.shape[1]+1)[::-1] / progress_arr.shape[1], cmap='winter', marker='x')
    plt.colorbar()
    plt.show()
    
    if res.success:
        print("Success!")
        u_guess[ITER][other] = res.x
    else: print("Failed:", res.message)

Finally, we do an optimization with all six parameters:

In [ ]:
def f_tot(x, p1, p2, spills):
    return 1 - score_spills(p1, p2, *spills, dx=x[0], dy=x[1], phi=x[2], \
                            ux=x[3], uxy=x[4], uy=x[5], 
                            nmax=15, alpha=0.05)

alignments = {ROOT: (0,0,0,0,0,0)}

for other in OTHERS:
    guess = np.array([*xyphi[-1][other], *u_guess[-1][other]])
    res = scipy.optimize.minimize(f_tot, guess, args=(ROOT, other, spills), method='Nelder-Mead', \
                                  options={'xatol': 1e-5}, callback=lambda xk: print(xk))
    
    if res.success:
        print("Success!")
        alignments[other] = res.x
    else: print("Failed:", res.message)

In [ ]:
np.set_printoptions(precision=4)

for p in PHONE_LIST:
    print('{}:'.format(p))
    print("Measured: ", alignments[p])
    print("Truth:", align_truth[p])
    print("Diff:", alignments[p] - align_truth[p])
    print()

In [ ]:
# output corrected coordinates

# TODO: GET RID OF THIS
alignments = align_truth

os.makedirs(os.path.join(DIR, 'align'), exist_ok=True)
for iphone in PHONE_LIST:
    
    for spl in spills:
        n_spl = 0
        for t in spl[iphone]:

            fspl = Spill.get_file(iphone, t, 'cluster')

            x_sensor = (fspl['x'] + 0.5 - RES_X / 2) * PIX_SIZE
            y_sensor = (fspl['y'] + 0.5 - RES_Y / 2) * PIX_SIZE
            
            x_lab, y_lab = sensor_map(x_sensor, y_sensor, *alignments[iphone])
            
            intersect_dict = {}
            for jphone in PHONE_LIST:
                if iphone == jphone: continue
                    
                xj, yj = inverse_map(x_lab, y_lab, *alignments[jphone])   
                
                intersect_dict[jphone] = (np.abs(xj) < LIM_X) & (np.abs(yj) < LIM_Y)
                
            np.savez('{}/align/{}_p{}_t{}.npz'.format(DIR, PREFIX, iphone, t), 
                     x=x_lab, 
                     y=y_lab,
                     t=fspl['t'], 
                     particles=fspl['particles'],
                     align=alignments[iphone], 
                     **intersect_dict)
            fspl.close()
        

# IV. Efficiency #

The general strategy for finding efficiency is simple enough.  We want to count the total number of coincidences on each sensor pair, subtract out the noise of different particles hitting the same region, and divide by the average number of particles that hit each sensor during the overlapping part of the frame.  Assuming the alignment has been done well, counting the coincidences should be easy.  For the background, if we have $N$ and $M$ uncorrelated hits on a pair of sensors, each with a distribution $P(\vec{r})$:

\begin{equation}
E(n_\mathrm{coinc})=NM \int d^2\vec{r_1} P(\vec{r_1}) \int d^2\vec{r_2} P(\vec{r_2}) \ \ \Theta\left(r_0 - \left|\vec{r_1} - \vec{r_2}\right|\right)
\end{equation}

where $\Theta$ is the Heaviside function and $r_0$ is a threshold distance which determines whether two points are considered a coincidence.  If $P(\vec{r})$ is roughly constant on the scale of $r_0$, we can treat it like a Delta function:

\begin{equation}
E(n_\mathrm{coinc}) \approx NM \int d^2\vec{r} P(\vec{r})^2
\end{equation}

and this integral can be evaluated numerically on the dataset.  In general, if we are finding coincidences among $n$ sensors with occupancies $N_1, \ldots N_n$, we can extend this as:

\begin{equation}
E(n_\mathrm{coinc}) \approx \prod_{i=1}^n N_i \int d^2\vec{r_1} P(\vec{r_1})^n
\end{equation}

However, for higher-order correlations, we need to take into account not just random noise, but also hits on two sensors coinciding with noise on others, etc.

Finally, to find the average number of particles hitting a single frame in the overlapping time window, we note that if the beam is on for the full duration of all frames (with occupancies $\{N_i\}$), which overlap for some fraction $f$ of the total frame duration:

\begin{equation}
n_\mathrm{overlap} \approx f \bar{N}
\end{equation}

On the other hand, if the beam starts or stops during one or more frames, but other frames receive the full intensity (i.e. the start or stop does not occur in the overlap window):

\begin{equation}
n_\mathrm{overlap} \approx f \max\{M_i\},
\qquad
\frac{N_{min}}{N_{max}} > f
\end{equation}

and if the start or stop *does* occur in the overlap window:

\begin{equation}
n_\mathrm{overlap} \approx \min\{N_i\},
\qquad
\frac{N_{min}}{N_{max}} < f
\end{equation}

In [ ]:
# get alignment params
aligns = {}
for p in PHONE_LIST:
    f0 = Spill.get_file(p, spills[0][p][0], filetype='align')
    aligns[p] = f0['align']
    f0.close()

In [ ]:
# iterator through overlapping frames
def gen_overlaps(spill, phones):

    phones = np.array(phones)
    times = [spill[p].copy() for p in phones]
    t_ijk = np.array([t.pop(0) for t in times]).astype(int)
    while True:
        overlap = calculate_overlap(FPS, *t_ijk)        
        if overlap:
            yield t_ijk, overlap

        # replace the earliest time, if possible
        p_earliest = np.argmin(t_ijk)
        if not times[p_earliest]: break
        t_ijk[p_earliest] = times[p_earliest].pop(0)
        
def gen_overlaps_single(spill, phones):
    phones = list(phones)
    times = {p:spill[p].copy() for p in phones}
    t_ijk = [times[p].pop(0) for p in phones]
    
    sort = np.argsort(t_ijk)
    phones = [phones[arg] for arg in sort]
    t_ijk = [t_ijk[arg] for arg in sort]
    
    initial_overlap = calculate_overlap(FPS, *t_ijk)
    for t, phone in zip(t_ijk[:-1], phones[:-1]):
        yield t, phone, 0
    yield t_ijk[-1], phones[-1], initial_overlap
    
    while True:
        # replace the earliest time, if possible
        t_ijk = t_ijk[1:]
        p_earliest = phones.pop(0)
        
        if not times[p_earliest]: break
        t_new = times[p_earliest].pop(0)
        
        t_ijk.append(t_new)
        phones.append(p_earliest)
        overlap = calculate_overlap(FPS, *t_ijk)
        
        yield t_new, p_earliest, overlap


In [ ]:
# find geometrical factors
X_MIN = {}
Y_MIN = {}

X_SIZE = {}
Y_SIZE = {}

for clist in COMBINATIONS[2:]:
    for c in clist:
    
        # get first frames
        spl = spills[0]
        f_ij = [Spill.get_file(p, spl[p][0], filetype='align') for p in c]

        x_offsets = [f['align'][0] for f in f_ij]
        y_offsets = [f['align'][1] for f in f_ij]
        max_phi, max_ux, max_uxy, max_uy = np.amax(np.abs([f['align'][2:] for f in f_ij]), axis=0)

        dx = np.diff(x_offsets)[0] / PIX_SIZE
        dy = np.diff(y_offsets)[0] / PIX_SIZE

        X_MIN[c] = max(x_offsets)-(RES_X*(max_ux+np.cos(max_phi)) + RES_Y*(max_uxy+np.sin(max_phi)))/2*PIX_SIZE
        Y_MIN[c] = max(y_offsets)-(RES_Y*(max_uy+np.cos(max_phi)) + RES_X*(max_uxy+np.sin(max_phi)))/2*PIX_SIZE

        X_SIZE[c] = np.ceil(RES_X*(max_ux+np.cos(max_phi))+RES_Y*(max_uxy+np.sin(max_phi)) - np.abs(dx)).astype(int)
        Y_SIZE[c] = np.ceil(RES_Y*(max_uy+np.cos(max_phi))+RES_X*(max_uxy+np.sin(max_phi)) - np.abs(dy)).astype(int)

        for f in f_ij: f.close()
        

In [ ]:
# locate the interior of the intersection in the root frame
INTERIOR = {}

bx = np.arange(0.5, RES_X, 1/np.sqrt(2))
by = np.arange(0.5, RES_Y, 1/np.sqrt(2))

bxi, byi = np.meshgrid(bx, by)

bxi = PIX_SIZE * (bxi - RES_X / 2).flatten()
byi = PIX_SIZE * (byi - RES_Y / 2).flatten()

for clist in COMBINATIONS[2:]:
    for c in clist:
        interior_coords = []
        for p in c:
            interior_x, interior_y = sensor_map(bxi, byi, *aligns[p])
            for pj in (c - set([p])):
                bxj, byj = inverse_map(interior_x, interior_y, *aligns[pj])
                
                intersect = (np.abs(bxj) < LIM_X) & (np.abs(byj) < LIM_Y)
                interior_x = interior_x[intersect]
                interior_y = interior_y[intersect]
            
            interior_x = ((interior_x - X_MIN[c]) / PIX_SIZE).astype(int)
            interior_y = ((interior_y - Y_MIN[c]) / PIX_SIZE).astype(int)
            interior_coords.append(interior_x + X_SIZE[c] * interior_y)
            
        interior_coords = np.unique(np.hstack(interior_coords))
        interior_mat = np.zeros((X_SIZE[c], Y_SIZE[c]), dtype=bool)
        interior_mat[interior_coords % X_SIZE[c], interior_coords // X_SIZE[c]] = True
        
        INTERIOR[c] = interior_mat
        
        plt.title(tuple([pk[:6] for pk in c]))
        plt.imshow(interior_mat, extent=[0, X_SIZE[c], 0, Y_SIZE[c]], cmap='gray', vmin=0)
        plt.show()
        
del bxi, byi, bxj, byj, interior_x, interior_y, interior_coords

In order to find $\int d^2\vec{r_1} P(\vec{r_1})^2$, we note that for a histogram (i.e. with Poisson statistics), $E(N(N-1)) = E(N^2) - E(N) = E((N-\bar{N})^2)) + E(N)^2 - E(N) = \sigma^2 + \mu^2 - \mu = \mu^2$

Similar methods can be employed with higher moments to achieve the result:

$\begin{equation}
\mu^n = E\left(N(N-1)\ldots(N-n+1)\right)
\end{equation}$

In [ ]:
# now test precision of effective area measurements
c0 = COMBINATIONS[-1][0]
bin_sz_options = (1,2,3,4,5,6,8,12,15)

profiles = {p: 0 for p in c0}
profile_temp = 0
for p in c0:
    for i,spl in enumerate(spills):
        for t in spl[p]:
            f = Spill.get_file(p, t, filetype='align')
            if not f['x'].size: continue
                
            intersect = np.ones(f['x'].size, dtype=bool)
            for iphone in c0:
                if iphone == p: continue
                intersect &= f[iphone]
            
            x = (f['x'][intersect] - X_MIN[c0]) / PIX_SIZE
            y = (f['y'][intersect] - Y_MIN[c0]) / PIX_SIZE

            # dither
            x += np.random.random(x.size) - 0.5
            y += np.random.random(x.size) - 0.5

            profile_temp += csr_matrix((np.ones(x.size), (x, y)), shape=(X_SIZE[c0], Y_SIZE[c0]))
            
            f.close()
            
        if not (i+1)%10 or i==len(spills)-1:
            profiles[p] += profile_temp.toarray()
            profile_temp = 0


plt_x = []
plt_y = []

for bin_sz in bin_sz_options:
    mu_2 = []
    for p in c0:
        xpad = (bin_sz - X_SIZE[c0] % bin_sz) % bin_sz
        ypad = (bin_sz - Y_SIZE[c0] % bin_sz) % bin_sz
        
        profile_pad = np.pad(profiles[p], ((0, xpad), (0, ypad)), 'constant')
        profile_ds = profile_pad.reshape(profile_pad.shape[0]//bin_sz, bin_sz, profile_pad.shape[1]//bin_sz, bin_sz).sum((1,3))

        mu_2.append((profile_ds * (profile_ds-1)).sum() / (profile_ds.sum()**2 * bin_sz**2))
    
    plt_x.append(bin_sz)
    plt_y.append(mu_2)
        
plt.figure(figsize=(5,5))

for ip, p in enumerate(c0):
    plt.plot(plt_x, [y[ip] for y in plt_y], label=p[:6], linestyle=':')
plt.plot(plt_x, np.mean(np.array(plt_y), axis=1), label='Mean')
#plt.semilogx()

plt.title(r'Calculating $\int P(\vec{r})^2 d\vec{r}$')
plt.xlabel(r'$n_{bins}$')
plt.ylabel(r'$1/A_{eff}$')
plt.legend()

plt.show()

del profile_temp, profile_pad, profile_ds

In [ ]:
# find the effective area

BEAM_PROFILE_CORR = {}
BEAM_PROFILE_VAR = {}

downsample = 12

for clist in COMBINATIONS[2:]:
    for c in clist:
        
        results = []
        
        bx = np.arange(X_SIZE[c])
        by = np.arange(Y_SIZE[c])
        bxx, byy = np.meshgrid(bx, by)
        
        for p in c:
            
            profile = 0
            profile_temp = 0
            
            for i,spl in enumerate(spills):
                for t in spl[p]:
                    f = Spill.get_file(p, t, filetype='align')
                    if not f['x'].size: continue
            
                    intersect = np.ones(f['x'].size, dtype=bool)
                    for iphone in c:
                        if iphone == p: continue
                        intersect &= f[iphone]
                        
                    x = (f['x'][intersect] - X_MIN[c]) / PIX_SIZE
                    y = (f['y'][intersect] - Y_MIN[c]) / PIX_SIZE
                        
                    # dither
                    x += np.random.random(x.size) - 0.5
                    y += np.random.random(y.size) - 0.5

                    profile_temp += csr_matrix((np.ones(x.size), (x, y)), shape=(X_SIZE[c], Y_SIZE[c]))

                    f.close()

                if not (i+1)%10 or i==len(spills)-1:
                    profile += profile_temp.toarray()
                    profile_temp = 0
        
            profile_sum = profile.sum()
            mu_n = 1
            p_results = []
        
            for i in range(len(c)):
                mu_n *= profile - i
                if i:
                    p_results.append(mu_n.sum() / profile_sum**(i+1))
                    
            results.append(p_results)
                

        BEAM_PROFILE_CORR[c] = np.mean(np.array(results), axis=0)
        BEAM_PROFILE_VAR[c] = np.var(results, axis=0) / len(results)
        
        # make a plottable version
        xpad = (downsample - X_SIZE[c] % downsample) % downsample
        ypad = (downsample - Y_SIZE[c] % downsample) % downsample
        
        profile_pad = np.pad(profile, ((0, xpad), (0, ypad)), 'constant')
        profile_ds = profile_pad.reshape(profile_pad.shape[0]//downsample, downsample, \
                                             profile_pad.shape[1]//downsample, downsample).sum((1,3))
        
        plt.title(list(map(lambda p: p[:6], c)))
        plt.imshow(profile_ds, cmap='plasma', extent=[0, profile_pad.shape[0], 0, profile_pad.shape[1]], vmin=0)
        plt.colorbar()
        plt.show()
    
        print('A_tot = {}'.format(np.count_nonzero(profile_ds) * downsample**2))
        print('A_eff = {}'.format(np.array(BEAM_PROFILE_CORR[c])**(-1/(np.arange(1, len(c))))))
        print('Fractional err:', np.sqrt(BEAM_PROFILE_VAR[c]) / BEAM_PROFILE_CORR[c])

del profile, profile_temp, profile_pad, profile_ds

In [ ]:
# at this point, we can measure the distribution of distances between hits
XY_HISTS = {}

survival_size = 100
xy_size = 15

# we only sample the interior to avoid edge effects
def find_edge(binary_image, n):
    binary_padded = np.pad(binary_image, ((n, n),(n,n)), 'constant')
    interior = binary_padded.copy()
    for roll in range(-n, n+1):
        for ax in (0,1):
            interior &= np.roll(binary_padded, roll, axis=ax) 
        
    return binary_image ^ interior[n:-n, n:-n]

xy_r = xy_size // 2
for c in COMBINATIONS[2]:
    survival_tot = np.zeros(survival_size)
    XY_HISTS[c] = np.zeros((xy_size, xy_size))
    
    hist_interior = INTERIOR[c] & np.logical_not(find_edge(INTERIOR[c], 2*xy_r))
    
    for spl in spills[:30]:
        tuple_c = tuple(sorted(c))
        for times, overlap in gen_overlaps(spl, tuple_c):
            f_ij = [Spill.get_file(p, t, filetype='align') for p, t in zip(tuple_c, times)]

            intersect1 = np.ones(f_ij[0]['x'].size, dtype=bool)
            intersect2 = np.ones(f_ij[1]['x'].size, dtype=bool)
            for iphone in c:
                if iphone != tuple_c[0]:
                    intersect1 &= f_ij[0][iphone]
                if iphone != tuple_c[1]:
                    intersect2 &= f_ij[1][iphone]
            
            x1 = (f_ij[0]['x'][intersect1] - X_MIN[c]) / PIX_SIZE
            y1 = (f_ij[0]['y'][intersect1] - Y_MIN[c]) / PIX_SIZE
            x2 = (f_ij[1]['x'][intersect2] - X_MIN[c]) / PIX_SIZE
            y2 = (f_ij[1]['y'][intersect2] - Y_MIN[c]) / PIX_SIZE
            
            for f in f_ij: f.close()
            
            # now cut out the edges on x1
            edge_cut = hist_interior[x1.astype(int), y1.astype(int)]
            
            x1 = x1[edge_cut]
            y1 = y1[edge_cut]
            
            xmin = 0
            xmax = X_SIZE[c]
            ymin = 0
            ymax = Y_SIZE[c]

            divx = (xmax - xmin) / NDIVS
            divy = (ymax - ymin) / NDIVS

            # make interlaced cells
            x1_cells, y1_cells = divide_points(x1, y1, NDIVS, limits=(xmin, xmax, ymin, ymax))
            x2_cells, y2_cells = divide_points(x2, y2, NDIVS+1, limits=(xmin - divx/2, xmax + divx/2, ymin-divy/2, ymax+divy/2))

            for i,j in np.ndindex(NDIVS, NDIVS):
                group1 = (x1_cells == i) & (y1_cells == j) 
                if not group1.sum():
                    continue

                group2 = ((x2_cells == i) | (x2_cells == i+1)) & ((y2_cells == j) | (y2_cells == j+1))
                if not group2.sum():
                    continue

                rsquared_min = np.amin((x1[group1] - x2[group2].reshape(-1,1))**2 \
                                       + (y1[group1] - y2[group2].reshape(-1,1))**2, axis=0)

                density = group2.sum() / (4 * divx * divy)
                survival = np.exp(-rsquared_min * np.pi * density)

                survival_tot += np.histogram(survival, bins=survival_size)[0]

                xy_vals = np.dstack([x1[group1] - x2[group2].reshape(-1,1), \
                                     y1[group1] - y2[group2].reshape(-1,1)]).reshape(-1, 2).transpose()

                XY_HISTS[c] += np.histogram2d(xy_vals[0], xy_vals[1], \
                                bins=(np.arange(-xy_r-0.5, xy_r + 1.5), np.arange(-xy_r - 0.5, xy_r + 1.5)))[0]

            XY_HISTS[c] -= x1.size * x2.size * BEAM_PROFILE_CORR[c]

    plt.figure(figsize=(9, 4))
    plt.subplot(121)
    plt.title('Min distances: ({}, {})'.format(tuple_c[0][:6], tuple_c[1][:6]))
    plt.xlabel('1-cdf')
    plt.hist(np.linspace(0, 1, survival_tot.size), weights=survival_tot, bins=survival_tot.size)
    
    xx, yy = np.meshgrid(np.arange(-xy_r, xy_r + 1), np.arange(-xy_r, xy_r+1))
    plt.subplot(122)
    plt.title('Hit separation: ({}, {})'.format(tuple_c[0][:6], tuple_c[1][:6]))
    plt.xlabel(r'$\Delta x$ (pixels)')
    plt.ylabel(r'$\Delta y$ (pixels)')
    
    plt.imshow(XY_HISTS[c], extent=[-xy_r-0.5, xy_r+0.5, -xy_r-0.5, xy_r+0.5], norm=LogNorm())
    plt.colorbar()
    plt.show()
    
del hist_interior

In [ ]:
CONV_SIZE_LIST = [(9,9),(7,7),(5,5),(5,5),(7,7),(5,5)]
CONV_SIZES = {COMBINATIONS[2][i]: CONV_SIZE_LIST[i] for i in range(len(COMBINATIONS[2]))} 

# now find the edge region to exclude
EDGE = {}
for clist in COMBINATIONS[2:]:
    for c in clist:
        EDGE[c] = find_edge(INTERIOR[c], max([max(conv) for conv in CONV_SIZES.values()]) // 2)

In [ ]:
# utility methods

def convolve_sparse(x, y, csize, shape):
    
    center = np.array(csize) // 2
    conv_x = []
    conv_y = []
    
    for idx, idy in np.ndindex(csize):
        ix = x + idx-center[0]
        iy = y + idy-center[1]
        valid = (ix > 0) & (iy > 0) & (ix < shape[0]) & (iy < shape[1])

        conv_x.append(ix[valid])
        conv_y.append(iy[valid])
    
    conv_x = np.hstack(conv_x)
    conv_y = np.hstack(conv_y)
    
    return csr_matrix((np.ones(conv_x.size), (conv_x, conv_y)), shape=shape)
    
    
def partition(iterable):
    collection = list(iterable)
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller
        
def superpartition(collection, n_sets):
    for idx in map(np.array, np.ndindex(*np.repeat(2**n_sets - 1, len(collection)))):
        include = (idx + 1) // 2**np.arange(n_sets).reshape(-1, 1) % 2 == 1
        if not np.all(include.sum(axis=1)): continue
        part = [list(np.array(collection)[include[i]]) for i in range(n_sets)]
        # make sure we only get each grouping once
        if part != sorted(part): continue
        yield part
        
def powerset(iterable, min_size=0, max_size=None):
    s = list(iterable)
    if not max_size: max_size = len(s)
    return map(frozenset, it.chain.from_iterable(it.combinations(s, r) for r in range(min_size, max_size+1)))


In [ ]:
# for each combination of phones, determine the fastest way to convolve
OPTIMAL_P = {}
for subset in powerset(PHONE_LIST, min_size=2):
    pi_best = None
    for pi in subset:
        total_conv = 0
        for pj in subset:
            if pi is pj: continue
            total_conv += np.product(CONV_SIZES[frozenset([pi, pj])])
            
        if pi_best == None or total_conv < min_conv:
            min_conv = total_conv
            pi_best = pi
            
    OPTIMAL_P[subset] = pi_best 
    
#HIT_FRAC = {}
#for c in COMBINATIONS[2]:
#    xy_hist = XY_HISTS[c][0] - BEAM_PROFILE_CORR[c][0]**XY_HISTS[c][1]
#    conv_size = np.array(CONV_SIZES[c])
    
#    xstart, ystart = np.array(xy_hist.shape) // 2 - conv_size // 2
#    xend, yend = np.array(xy_hist.shape) // 2 + conv_size // 2 + 1
    
#    HIT_FRAC[c] = xy_hist[xstart:xend, ystart:yend].sum() / xy_hist.sum()
#for k,v in OPTIMAL_P.items():
#    print([ki[:6] for ki in k], v)

In [ ]:
# get effective convolution sizes as well
CONV_FACTORS = {}

min_convolutions = {}

for pi in PHONE_LIST:
    p_other = PHONE_LIST.copy()
    p_other.remove(pi)
    for s in map(list, powerset(p_other, min_size=1)):
        s_conv = np.array([CONV_SIZES[frozenset([pi, si])] for si in s])
        min_conv_idx = np.argmin(np.product(s_conv, axis=1))
        min_conv = s_conv[min_conv_idx]
                                
        conv_tot = 0
        
        #printarr = np.zeros(min_conv)
        # iterate over positions for particle to be incident on min_conv sensor
        for ixy in map(lambda a: np.array(a) - (min_conv // 2), np.ndindex(tuple(min_conv))):
            conv_contribs = [1]
            
            # now multiply by probabilities for particle to be in convolution windows on other sensors
            for s_idx, pj in enumerate(s):
                if s_idx == min_conv_idx: continue
                
                xy_idx = frozenset([s[min_conv_idx], pj])
                xy_hist = XY_HISTS[xy_idx]
                xy_size = np.array(CONV_SIZES[xy_idx]) 
                
                xstart, ystart = np.array(xy_hist.shape) // 2 + ixy - s_conv[s_idx] // 2
                xend = xstart + s_conv[s_idx][0]
                yend = ystart + s_conv[s_idx][1]
                
                hist_cut = (xy_hist[xstart:xend, ystart:yend]).sum() / xy_hist.sum()

                conv_contribs.append(hist_cut)
                        
            # mean of upper (perfect correlation) and lower (no correlation) bounds
            conv_tot += (np.product(conv_contribs) + min(conv_contribs)) / 2
            #printarr[tuple(ixy + (min_conv // 2))] = min(conv_contribs)
        
        CONV_FACTORS[(pi, frozenset(s))] = conv_tot
        min_convolutions[(pi, frozenset(s))] = np.product(min_conv)
        
        #print(printarr)
        #print(conv_tot)
        
for clist in COMBINATIONS[2:]:
    for c in map(frozenset, clist):
        pi = OPTIMAL_P[c]
        pj_all = c - set([pi])
        
        print(pi[:6], [p[:6] for p in pj_all], min_convolutions[(pi, frozenset(pj_all))], CONV_FACTORS[(pi, frozenset(pj_all))])
        

In [ ]:
NOISE_INTERSECT = {}

for clist in COMBINATIONS[2:]:
    for c in clist:
        
        results = []
        
        bx = np.arange(X_SIZE[c])
        by = np.arange(Y_SIZE[c])
        bxx, byy = np.meshgrid(bx, by)
        
        for p in c:
    
            bxx_sensor, byy_sensor = inverse_map((bxx*PIX_SIZE + X_MIN[c]).flatten(), \
                                                 (byy*PIX_SIZE + Y_MIN[c]).flatten(), \
                                                 *aligns[p])

            # now map to coordinates of the noise histograms
            cut = (np.abs(bxx_sensor) < RES_X / 2 * PIX_SIZE) & (np.abs(byy_sensor) < RES_Y / 2 * PIX_SIZE)

            bxx_sensor = (bxx_sensor / PIX_SIZE + RES_X / 2).astype(int)[cut]
            byy_sensor = (byy_sensor / PIX_SIZE + RES_Y / 2).astype(int)[cut]

            values_cut = NOISE[p][bxx_sensor, byy_sensor]

            template = np.zeros((X_SIZE[c], Y_SIZE[c]))
            template[bxx.flatten()[cut], byy.flatten()[cut]] = values_cut
            template *= (INTERIOR[c] & np.logical_not(EDGE[c]))

            NOISE_INTERSECT[(c, p)] = template

            #plt.imshow(template, cmap='viridis', extent=[0, X_SIZE[c], 0, Y_SIZE[c]])
            #plt.colorbar()
            #plt.title('Noise: {}'.format(p[:6]))
            #plt.show()
            
    

In [ ]:
%matplotlib inline

# this should work faster than a standard convolution for a sparse matrix
eff_estimates = {}
all_diffs = {}
noise_diffs = {}
single_diffs = {}
    
for clist in COMBINATIONS[2:]:
    for c in map(frozenset, clist):
        
        n_all = []
        n_single = []
        
        #q = []
        
        truth_all = []
        truth_single = []
        
        for n_spills, spl in enumerate(spills):
            print("{:.2f}%".format(100*n_spills/len(spills)), end="\r")
            
            all_spl = 0
            single_spl = 0
            
            #q_spl = 0
            
            truth_all_spl = 0
            truth_single_spl = 0
            
            # keep a cache of results
            n_coinc = {}
            n_hits_interior = {}
            n_hits_corr = {}
            first_last = {p: True for p in c}
            conv_ij = {p: {} for p in c}
            
            truth_particles = {}
            
            # one way to propagate the error in the beam profile estimate
            #profile_corr = [np.random.normal(corr, np.sqrt(var)) for corr, var \
            #                in zip(BEAM_PROFILE_CORR[c], BEAM_PROFILE_VAR[c])]

            for t, p, overlap in gen_overlaps_single(spl, c):
                
                # clear out old entries
                conv_ij[p] = {}
                
                f = Spill.get_file(p, t, filetype='align')
                
                intersect = np.ones(f['x'].size, dtype=bool)
                for iphone in c:
                    if iphone == p: continue
                    intersect &= f[iphone]
                
                x_new = ((f['x'][intersect] - X_MIN[c]) / PIX_SIZE).astype(int)
                y_new = ((f['y'][intersect] - Y_MIN[c]) / PIX_SIZE).astype(int)
                                
                interior = np.logical_not(EDGE[c][x_new, y_new])
                
                n_coinc[frozenset([p])] = x_new.size
                n_hits_interior[p] = interior.sum()
                n_hits_corr[p] = n_hits_interior[p] - NOISE_INTERSECT[(c,p)].sum()
                first_last[p] = (t in np.array(spl[p])[[0,-1]])

                particles_intersect = f['particles'][intersect[:f['particles'].size]]
                truth_particles[p] = particles_intersect[interior[:particles_intersect.size]] \
                                    if p == OPTIMAL_P[c] else particles_intersect
                truth_single_spl += interior[:particles_intersect.size].sum() / len(c)
                                
                f.close()
                
                for subc in powerset(c, min_size=2):
                    if not p in subc: continue
                    
                    # at the very least, we make the convolved matrix for p
                    pi = OPTIMAL_P[subc]
                    
                    if not pi in conv_ij[p]:
                        if p == pi:
                            conv_ij[p][pi] = convolve_sparse(x_new[interior], y_new[interior], \
                                                           (1,1), shape=(X_SIZE[c], Y_SIZE[c]))
                        else: 
                            conv_ij[p][pi] = convolve_sparse(x_new, y_new, CONV_SIZES[frozenset([pi, p])],
                                                           shape=(X_SIZE[c], Y_SIZE[c]))
                    
                    
                    pj_all = subc - set([pi])
                    if not all([pi in conv_ij[pij] for pij in subc]): continue
                        
                    sparse_i = conv_ij[pi][pi].copy()
                    
                    for pj in pj_all:
                        sparse_i = sparse_i.multiply(conv_ij[pj][pi])
                
                    # now we calculate the noise
                    noise_tot = 0
                    for part in partition(subc):
                        
                        order = len(part)
                        if order == 1: continue # this is the signal term
                            
                        noise_contribution = BEAM_PROFILE_CORR[c][order-2] #(RES_X * RES_Y) ** float(1-order) #
                        
                        for s in map(frozenset, part):
                            if s == frozenset([pi]):
                                noise_contribution *= n_hits_interior[pi]
                            else:
                                noise_contribution *= n_coinc[s]
                            
                            if not pi in s:
                                noise_contribution *= CONV_FACTORS[(pi, s)]
                        
                        noise_tot += noise_contribution
                    
                    n_coinc[subc] = (sparse_i.sum() - noise_tot) #/ hit_frac
                
                if not overlap: continue
                
                all_spl += n_coinc[c]
                #q_spl += sparse_i.sum()
                truth_all_spl += len(set.intersection(*map(set, truth_particles.values())))
                
                if np.any(list(first_last.values())):

                    if min(n_hits_corr.values()) <= 0 \
                    or min(n_hits_corr.values()) / max(n_hits_corr.values()) > overlap:
                        if np.all(list(first_last.values())):
                            # either we're missing a frame, or this is a statistical fluke
                            single_spl += overlap * max(n_hits_corr.values())
                        else:
                            single_spl += overlap * np.mean([n_hits_corr[p] for p in c if not first_last[p]])
                    else:
                        single_spl += min(n_hits_corr.values())

                else:
                    single_spl += overlap * np.mean(list(n_hits_corr.values()))
                    
            n_all.append(all_spl)
            n_single.append(single_spl)
            
            #q.append(q_spl)
            
            truth_all.append(truth_all_spl)
            truth_single.append(truth_single_spl)
            
        print("100% ", end="\r")
        print("all:", np.mean(n_all))
        print("single", np.mean(n_single))
        print("truth all", np.mean(truth_all))
        print("truth single", np.mean(truth_single))
        #print("q", np.mean(q))
        #print("noise", np.mean(q) - np.mean(n_all))
        
        # now find an unbiased ratio estimator (to first order) for each spill

        eff_estimates[c] = np.array(n_all) / np.array(n_single)
        all_diffs[c] = np.array(n_all) / np.array(truth_all) - 1
        
        plt.figure(figsize=(12, 4))
        plt.suptitle(tuple(map(lambda p: p[:6], c)))
        plt.subplot(121)
        plt.title('Efficiency by spill')
        plt.hist(eff_estimates[c], bins=35)
        plt.xlabel(r'$\epsilon$')
        
        plt.subplot(122)
        plt.title('Numerator')
        plt.hist(all_diffs[c], bins=35)

        plt.show()

In [ ]:
ftruth = np.load('{}/{}_truth.npz'.format(DIR, PREFIX))

weighted_efficiencies = []
weighted_eff_vars = []

for ic, clist in enumerate(COMBINATIONS[2:]):
    means = []
    variances = []
    for c in clist:
        mean_c = np.mean(eff_estimates[c])
        var_c = np.var(eff_estimates[c]) / len(eff_estimates[c])
        means.append(mean_c)
        variances.append(var_c)
        print("{}: {:.6f} +/- {:.6f}".format(tuple(map(lambda p: p[:6], c)), mean_c, np.sqrt(var_c)))
    
    # now do a weighted average
    means = np.array(means)
    variances = np.array(variances)
    
    eff = np.sum(means / variances) / np.sum(1 / variances)
    
    # because of systematic error, we find the weighted sample variance here
    if means.size == 1:
        eff_var = variances[0]
    else:
        eff_var = np.sum((means-eff)**2 / variances) / np.sum(1 / variances)
    
    print("Total: {:.7f} +/- {:.7f}".format(eff, np.sqrt(eff_var)))
    print("Truth: {:.7f}".format(np.dot(ftruth['eff']**(ic+2), PARTICLES) / np.dot(ftruth['eff'], PARTICLES)))
    print()
    weighted_efficiencies.append(eff)
    weighted_eff_vars.append(eff_var)
    
ftruth.close()

The $n$-point efficiency estimates generated here are just:

$\begin{equation}
\epsilon^{(n)} = \frac{\sum_{i=1}^{k}\epsilon_i^n \phi_i}{\sum_{i=1}^{k}\epsilon_i \phi_i}
\end{equation}$

where $k$ is the number of particles, and $\phi_i$ are the fluxes for each particle.  Dividing by the total flux $\sum_{i=1}^k \phi_i$ on both top and bottom:

$\begin{equation}
\epsilon^{(n)} = \frac{\sum_{i=1}^{k}\epsilon_i^n f_i}{\sum_{i=1}^{k}\epsilon_i f_i}
\end{equation}$

where $f_i$ denotes the known fractional flux of each particle.  This is a nonlinear system of equations which can be solved for each $\epsilon_i$.

In [ ]:
if len(PARTICLES) == 1:
    # just do a weighted average, taking the correct power of the efficiency
    sum_wx = 0
    sum_w = 0
    for i in range(len(weighted_efficiencies)):
        b = 1/(i+1)
        eff = weighted_efficiencies[i]**b
        eff_var = weighted_eff_vars[i] * b**2 / eff ** (2*i)
        
        sum_wx += eff / eff_var
        sum_w += 1 / eff_var
        
    print("eff: {:.7} +/- {:.7}".format(sum_wx/sum_w, np.sqrt(1/sum_w)))

In [ ]:
%matplotlib notebook

COLZ = ['r','g','b']

if len(PARTICLES) == 2:
    A = np.linspace(0,1,100)
    X, Y = np.meshgrid(A,A)
    
    plt.figure(figsize=(7,7))

    plt.xlabel(r'$\epsilon_1$')
    plt.ylabel(r'$\epsilon_2$')

    plt.xlim(0,1)
    plt.ylim(0,1)
    
    contours = []
    labels = []
    
    for i, eff in enumerate(weighted_efficiencies):
        
        def fn(x, y):
            X = np.moveaxis(np.array([x, y]), 0, 2)
            return np.dot(X**(i+2), PARTICLES) / np.dot(X, PARTICLES)
        
        contours.append(plt.contour(X, Y, fn(X, Y), [eff], colors=COLZ[i]))
        labels.append('{} phones'.format(i+2))
        
    plt.legend(map(lambda cont: cont.legend_elements()[0][0], contours), labels)
    plt.show()
    
elif len(PARTICLES) == 3:
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.gca(projection='3d')

    ax.set_xlabel(r'$\epsilon_1$')
    ax.set_ylabel(r'$\epsilon_2$')
    ax.set_zlabel(r'$\epsilon_3$')

    ax.set_xlim(0,1)
    ax.set_ylim(1,0)
    ax.set_zlim(0,1)
    
    u = np.linspace(0, np.pi / 2, 400)
    v = np.linspace(0, np.pi / 2, 400)
    
    x0 = np.outer(np.sin(v), np.cos(u))
    y0 = np.outer(np.sin(v), np.sin(u))
    z0 = np.outer(np.cos(v), np.ones(u.size))
    
    X = np.moveaxis(np.array([x0, y0, z0]), 0, 2)
    

    r = []
    
    for i,eff in enumerate(weighted_efficiencies):
        
        r.append((eff * np.dot(X, PARTICLES) / np.dot(X**(i+2), PARTICLES))**(1/(i+1)))
        
    r = np.array(r)
    args = np.argmax(r, axis=0) 
    
    # slice on the max r values 
    idx = list(np.ogrid[[slice(r.shape[ax]) for ax in range(r.ndim) if ax != 0]])
    idx.insert(0, args)
    idx = tuple(idx)
    
    r = r[idx]
    c = np.array([np.full_like(x0, c, dtype=str) for c in COLZ])[idx]
    
    e1 = r * x0
    e2 = r * y0
    e3 = r * z0
    
    ax.plot_surface(e1, e2, e3, rstride=1, cstride=1, facecolors=c)
    
    plt.figure(figsize=(8,8))
    
    X, Y = np.meshgrid(np.linspace(0, np.pi/2, args.shape[0]), 
                       np.linspace(0, np.pi/2, args.shape[1]))
    
    # create a custom colormap
    cm = LinearSegmentedColormap.from_list('rgb', [(1, 0, 0), (0, 1, 0), (0, 0, 1)], N=3)
    plt.imshow(args[::-1,::-1], cmap=cm, 
               origin='lower', extent=[0, np.pi/2, 0, np.pi/2])
    contour = plt.contour(X, Y, r[::-1, ::-1], cmap='gray')
    plt.clabel(contour, fontsize=8)
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'$\theta$')

In [ ]:
guess = (.3, .7)

if len(PARTICLES) == 2:
    plt.figure(figsize=(7,7))

    plt.xlabel(r'$\epsilon_1$')
    plt.ylabel(r'$\epsilon_2$')

    plt.xlim(guess[0]-0.02, guess[0]+0.02)
    plt.ylim(guess[1]-0.002, guess[1]+0.002)
    
    for i, eff, eff_var in zip(range(len(weighted_efficiencies)), weighted_efficiencies, weighted_eff_vars):
        
        def fn(x, y):
            X = np.moveaxis(np.array([x, y]), 0, 2)
            return np.dot(X**(i+2), PARTICLES) / np.dot(X, PARTICLES)
        
        if eff_var:
            plt.contour(X, Y, fn(X, Y), [eff-np.sqrt(eff_var), eff, eff+np.sqrt(eff_var)], colors=COLZ[i])
        else:
            plt.contour(X, Y, fn(X, Y), [eff], colors=COLZ[i])
        
    plt.show()

In [ ]:
guess = (1.07, 1.21)

def f(phitheta):
    phi, theta = phitheta
    
    x0 = np.sin(theta) * np.cos(phi)
    y0 = np.sin(theta) * np.sin(phi)
    z0 = np.cos(theta)
    
    X = np.array([x0, y0, z0])

    f2 = np.dot(X, PARTICLES) / np.dot(X**2, PARTICLES) * weighted_efficiencies[0]
    f3 = np.dot(X**2, PARTICLES) / np.dot(X**3, PARTICLES) * weighted_efficiencies[1] / weighted_efficiencies[0]
    f4 = np.dot(X**3, PARTICLES) / np.dot(X**4, PARTICLES) * weighted_efficiencies[2] / weighted_efficiencies[1]
    
    return np.array([f3 - f2, f4 - f2])

guess_compl = np.pi/2 - np.array(guess)

phi, theta = scipy.optimize.fsolve(f, guess_compl)
print('\u03C6 = {:.4f}'.format(np.pi/2 - phi))
print('\u03B8 = {:.4f}'.format(np.pi/2 - theta))
print()

x0 = np.sin(theta) * np.cos(phi)
y0 = np.sin(theta) * np.sin(phi)
z0 = np.cos(theta)

X = np.array([x0, y0, z0])

r = np.dot(X, PARTICLES) / np.dot(X**2, PARTICLES) * weighted_efficiencies[0]

e1 = r * np.sin(theta) * np.cos(phi)
e2 = r * np.sin(theta) * np.sin(phi)
e3 = r * np.cos(theta)

print('\u03B51 = {:.5f}'.format(e1))
print('\u03B52 = {:.5f}'.format(e2))
print('\u03B53 = {:.5f}'.format(e3))